In [1]:
import pandas as pd
import random
import sys
sys.path.append('../..')

from model import run
from model import plot_utils as pu
from model import processing
from model.amm.omnipool_amm import OmnipoolState
from model.amm.stableswap_amm import StableSwapPoolState
from model.amm.agents import Agent
from model.amm.trade_strategies import random_swaps
from model.amm.global_state import GlobalState, swap, add_liquidity, migrate, migrate_lp, create_sub_pool
from model.amm import omnipool_amm as oamm

# same seed, same parameters = same simulation result
random.seed(42)

# set up the state

initial_state = GlobalState(
    pools = {
        "Omnipool": OmnipoolState(
            tokens={
                'HDX': {'liquidity': 1000000, 'LRNA_price': 4},
                'USD': {'liquidity': 1000000, 'LRNA_price': 100},  # this means 100 LRNA = 1 USD.
                'USDA': {'liquidity': 1000000, 'LRNA_price': 100},  # alternatively, you can specify 'LRNA': 100000000
                'USDB': {'liquidity': 1000000, 'LRNA_price': 100},  # same result.
                'USDC': {'liquidity': 1000000, 'LRNA_price': 100},
                'DAI': {'liquidity': 1000000, 'LRNA_price': 100},
                'R1': {'liquidity': 1000000, 'LRNA_price': 151},
                'R2': {'liquidity': 3000000, 'LRNA_price': 152},
                'R3': {'liquidity': 2000000, 'LRNA_price': 153},
                'R4': {'liquidity': 4000000, 'LRNA_price': 154},
            },
            lrna_fee=0,  # these can also be FeeMechanism type
            asset_fee=0
        )
    },
    agents = {
        'Trader': Agent(
            holdings={'HDX': 10000, 'USD': 10000, 'R1': 10000, 'R2': 10000},
        ),
        'LP': Agent(
            holdings={'DAI': 1000000, 'R1': 1000000}, 
        )
    }
).create_sub_pool(
    pool_id='Omnipool',
    sub_pool_id='usd_stableswap',
    tkns_migrate=['USDA', 'USDB', 'USDC'], 
    amplification=10,
    trade_fee=0
).create_sub_pool(
    pool_id='Omnipool',
    sub_pool_id='r1_stableswap',
    tkns_migrate=['R1', 'R2', 'R3', 'R4'],
    amplification=100,
    trade_fee=0.003
)
# print(initial_state)

In [2]:
print(initial_state.pools['Omnipool'].liquidity['usd_stableswap'])

300000000.0


In [8]:
migrate_state = migrate(
    old_state=initial_state,
    pool_id='Omnipool',
    sub_pool_id='usd_stableswap',
    tkn_migrate='DAI'
)

# sell an asset
buy_quantity = 1000
tkn_buy = 'DAI'
tkn_sell = 'R1'
initial_agent = migrate_state.agents['Trader']
initial_agent.holdings['DAI'] = 1000000

new_state = swap(
    old_state=migrate_state,
    pool_id='Omnipool',
    agent_id='Trader',
    tkn_buy=tkn_buy,
    tkn_sell=tkn_sell,
    buy_quantity=buy_quantity
)
# print(new_state)
new_omnipool = new_state.pools["Omnipool"]
new_agent = new_state.agents['Trader']

sell_quantity = initial_agent.holdings[tkn_sell] - new_agent.holdings[tkn_sell]
smaller_initial_trade_state, before_trade_agent = oamm.swap(
    old_state=migrate_state.pools['Omnipool'],
    old_agent=migrate_state.agents['Trader'],
    tkn_buy=tkn_buy,
    tkn_sell=tkn_sell,
    buy_quantity=sell_quantity / 1000
)
print(f'sell quantity {sell_quantity}')
smaller_after_trade_state, after_trade_agent = oamm.swap(
    old_state=new_omnipool,
    old_agent=new_agent,
    tkn_buy=tkn_buy,
    tkn_sell=tkn_sell,
    buy_quantity=sell_quantity / 1000
)
print(before_trade_agent)
print(initial_agent)
print(smaller_initial_trade_state.fail)
spot_price_before = (
    (initial_agent.holdings[tkn_sell] - before_trade_agent.holdings[tkn_sell]) /
    (before_trade_agent.holdings[tkn_buy] - initial_agent.holdings[tkn_buy])
)
spot_price_after = (
    (after_trade_agent.holdings[tkn_sell] - new_agent.holdings[tkn_sell]) /
    (new_agent.holdings[tkn_buy] - after_trade_agent.holdings[tkn_buy])
)
execution_price = sell_quantity / (new_agent.holdings[tkn_buy] - initial_agent.holdings[tkn_buy])
print(f'spot price before: {spot_price_before}')
print(f'execution price: {execution_price}')
print(f'spot price after: {spot_price_after}')

sell quantity 653.97974844928310609140745044044465393302966891553
Agent: Trader
********************************
trade strategy: None
holdings: (

    *HDX*: 10000

    *USD*: 10000

    *R1*: 9999.5724456513

    *R2*: 10000

    *USDC*: 0

    *DAI*: 1000000.6539797485

    *R3*: 0

    *R4*: 0

    *USDB*: 0

    *USDA*: 0

    *usd_stableswap*: 0.0

    *r1_stableswap*: 0.0
)

Agent: Trader
********************************
trade strategy: None
holdings: (

    *HDX*: 10000

    *USD*: 10000

    *R1*: 10000

    *R2*: 10000

    *USDC*: 0

    *DAI*: 1000000

    *R3*: 0

    *R4*: 0

    *USDB*: 0

    *USDA*: 0

    *usd_stableswap*: 0

    *r1_stableswap*: 0
)


spot price before: 0.65377307133428602959054965558991143977505013551721
execution price: 0.65397974844928310609140745044044465393302966891553
spot price after: 0.65418676165054067101024104820894016387400504154996


In [9]:
# reverse that operation
sell_quantity = initial_state.agents['Trader'].holdings[tkn_sell] - new_state.agents['Trader'].holdings[tkn_sell]
sell_state = swap(
    old_state=initial_state,
    pool_id='Omnipool',
    agent_id='Trader',
    tkn_buy=tkn_buy,
    tkn_sell=tkn_sell,
    sell_quantity=sell_quantity
)
print(sell_state)

global state 
pools: 

    Omnipool: Omnipool
    ********************************
    tvl cap: inf
    lrna fee: 0%
    asset fee: 0%
    asset pools: (
    
        *HDX*
        asset quantity: 1000000.0
        lrna quantity: 4000000.0
        USD price: 0.04
        tvl: 40000.0
        weight: 4000000.0/2033000000.0 (0.001967535661583866)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
        *USD*
        asset quantity: 1000000.0
        lrna quantity: 100000000.0
        USD price: 1.0
        tvl: 1000000.0
        weight: 100000000.0/2033000000.0 (0.04918839153959666)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
        *DAI*
        asset quantity: 999000.7488534335
        lrna quantity: 100100025.06481734
        USD price: 1.0020015018
        tvl: 1001000.2506481734
        weight: 100100025.06481734/2033000000.0 (0.049237592260116744)
        weight cap: 1.0
        tota

In [ ]:
# chain operations together
partial_lp_state = add_liquidity(
    old_state=sell_state,
    pool_id='Omnipool',
    agent_id='LP',
    tkn_add='USDA',
    quantity=initial_state.agents['LP'].holdings['USDA']
)
migrate_state = migrate(
    old_state=partial_lp_state,
    pool_id='Omnipool',
    sub_pool_id='Stableswap',
    tkn_migrate='USDA'
)
migrate_lp_state = migrate_lp(
    old_state=migrate_state,
    pool_id='Omnipool',
    sub_pool_id='Stableswap',
    agent_id='LP',
    tkn_migrate='USDA'
)
print('Migrate State:', migrate_state)

print('Final State:', migrate_lp_state)